Validating the Qiskit version

In [128]:
import qiskit

qiskit.version.get_version_info()

'2.2.0'

In [129]:
import qiskit_aer

qiskit_aer.version.get_version_info()



'0.17.2'

In [130]:
from qiskit_aer import AerSimulator


Testing the sample

In [131]:
# ghz_state_solution.py

from qiskit import QuantumCircuit,transpile
from qiskit_aer import Aer

def build_ghz_circuit(num_qubits):
    qc = QuantumCircuit(num_qubits, num_qubits)

    # Apply Hadamard on first qubit
    qc.h(0)

    # Apply CNOT chain from first qubit to all others
    for i in range(1, num_qubits):
        qc.cx(0, i)

    # Measure all qubits
    qc.measure(range(num_qubits), range(num_qubits))

    return qc
def simulate_circuit(qc, shots=1024):
    sim = Aer.get_backend("qasm_simulator")
    transpiled = transpile(qc, backend=sim)
    result = sim.run(transpiled, shots=shots).result()
    counts = result.get_counts()
    return counts


num_qubits = 3  # try 4 as well
qc = build_ghz_circuit(num_qubits)
print("Generated GHZ circuit:")
print(qc.draw())

counts = simulate_circuit(qc, shots=1024)
print("Measurement counts:", counts)

Generated GHZ circuit:
     ┌───┐             ┌─┐   
q_0: ┤ H ├──■────■─────┤M├───
     └───┘┌─┴─┐  │  ┌─┐└╥┘   
q_1: ─────┤ X ├──┼──┤M├─╫────
          └───┘┌─┴─┐└╥┘ ║ ┌─┐
q_2: ──────────┤ X ├─╫──╫─┤M├
               └───┘ ║  ║ └╥┘
c: 3/════════════════╩══╩══╩═
                     1  0  2 
Measurement counts: {'111': 532, '000': 492}


Using Sample - implemting the Shor's Algorithm

In [132]:
# shor_from_scratch_qiskit_task.py

import math
import numpy as np
from fractions import Fraction
from collections import Counter

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library import QFT, UnitaryGate


# ============================================================
# Section 1: Helper math functions (already implemented)
# ============================================================

def is_coprime(a, b):
    return math.gcd(a, b) == 1

def continued_fraction_expansion(x, max_den=1_000_000):
    frac = Fraction(x).limit_denominator(max_den)
    return frac.numerator, frac.denominator

def fraction_from_phase(phase, qcount):
    y_over_2q = phase / (2**qcount)
    num, den = continued_fraction_expansion(y_over_2q, max_den=2**qcount)
    return num, den

def get_most_likely_result(counts):
    measured_str = Counter(counts).most_common(1)[0][0]
    return int(measured_str, 2), measured_str


Using Sample - creating the Quantum Blocks


In [133]:
# ============================================================
# Section 2: Quantum building blocks (tasks to complete)
# ============================================================

def build_multiplication_mod_matrix(a, N):
    """
    Task: Build the modular multiplication matrix U_a of size 2^n x 2^n.
    |x> -> |(a*x) mod N>  for x < N,
    |x> -> |x>            for x >= N
    """
    return None  # <-- replace with matrix construction


def controlled_powered_unitary_gate(U_matrix, power):
    """
    Task: Return a UnitaryGate for U^power.
    Hint: use numpy.linalg.matrix_power and UnitaryGate.
    """
    return None  # <-- replace with gate construction

Using Sample - Shor's order finding

In [134]:
# ============================================================
# Section 3: Shor's order finding (main quantum part)
# ============================================================

def shor_order_finding(N, a, qcount=None, shots=1024, seed_sim=42, verbose=True):
    if not (1 < a < N) or math.gcd(a, N) != 1:
        raise ValueError("a must be 1 < a < N and coprime to N")

    n = math.ceil(math.log2(N))
    if qcount is None:
        qcount = 2 * n

    # Task: Build U_a matrix
    U = build_multiplication_mod_matrix(a, N)

    total_qubits = qcount + n
    qc = QuantumCircuit(total_qubits, qcount)

    counting_qubits = list(range(qcount))
    target_qubits = list(range(qcount, qcount + n))

    # Initialize target register to |1>
    qc.x(target_qubits[0])

    # Apply Hadamards to counting register
    qc.h(counting_qubits)

    # Task: Apply controlled-U^(2^j) gates for each counting qubit
    # for j in range(qcount):
    #     power = 2**j
    #     U_pow_gate = controlled_powered_unitary_gate(U, power).control()
    #     qc.append(U_pow_gate, [counting_qubits[j]] + target_qubits)

    # Task: Apply inverse QFT on counting register
    # invqft = inverse_qft_circuit(qcount)
    # qc.compose(invqft, qubits=counting_qubits, inplace=True)

    # Task: Add measurement of counting register
    # qc.measure(counting_qubits, list(range(qcount)))

    # Running on simulator (already implemented)
    sim = AerSimulator(seed_simulator=seed_sim)
    transpiled = transpile(qc, sim, seed_transpiler=seed_sim)
    result = sim.run(transpiled, shots=shots).result()
    counts = result.get_counts()

    y_int, y_str = get_most_likely_result(counts)
    if verbose:
        print(f"Most frequent measurement (counting register): {y_str} -> int {y_int}")

    s, r_candidate = fraction_from_phase(y_int, qcount)
    if verbose:
        print(f"Continued fraction approx: s={s}, r_candidate={r_candidate}")

    r = r_candidate
    for mult in range(1, 11):
        r_try = r * mult
        if r_try != 0 and pow(a, r_try, N) == 1:
            if verbose:
                print(f"Found order r = {r_try}")
            return r_try, counts
    return None, counts

Using Sample - Classical Post Processing

In [135]:
# ============================================================
# Section 4: Classical postprocessing (already given)
# ============================================================

def shor_factor(N, shots=1024, tries=5, verbose=True):
    if N % 2 == 0:
        return 2, N // 2
    for attempt in range(tries):
        a = np.random.randint(2, N-1)
        if math.gcd(a, N) != 1:
            d = math.gcd(a, N)
            return d, N // d

        if verbose:
            print(f"Attempt {attempt+1}: trying base a = {a}")

        r, counts = shor_order_finding(N, a, shots=shots, verbose=verbose)
        if r is None or r % 2 != 0:
            continue

        apow = pow(a, r // 2, N)
        candidate1 = math.gcd(apow - 1, N)
        candidate2 = math.gcd(apow + 1, N)

        if candidate1 not in [1, N]:
            return candidate1, N // candidate1
        if candidate2 not in [1, N]:
            return candidate2, N // candidate2
    return None

Testing the Shor's Algorithm implementation

N= 15

In [136]:
# -------------------------
# Demo N=15
# -------------------------

if __name__ == "__main__":
    N = 15
    print(f"Running Shor's algorithm to factor N={N} with 1024 shots...")
    factors = shor_factor(N, shots=1024, tries=5, verbose=True)
    if factors:
        p, q = factors
        print(f"\n*** Found nontrivial factors: {p} × {q} = {N} ***")
    else:
        print("\nNo factors found. Try re-running (probabilistic algorithm).")

Running Shor's algorithm to factor N=15 with 1024 shots...

*** Found nontrivial factors: 5 × 3 = 15 ***


Testing for N= 21

In [137]:
# -------------------------
# Demo N=21
# -------------------------
if __name__ == "__main__":
    N = 21   # <<-- changed here
    print(f"Running Shor's algorithm to factor N={N} with 1024 shots...")
    factors = shor_factor(N, shots=1024, tries=5, verbose=True)
    if factors:
        p, q = factors
        print(f"\n*** Found nontrivial factors: {p} × {q} = {N} ***")
    else:
        print("\nNo factors found. Try re-running (probabilistic algorithm).")

Running Shor's algorithm to factor N=21 with 1024 shots...

*** Found nontrivial factors: 3 × 7 = 21 ***
